In [ ]:
from google.colab import drive, userdata

comet_key = userdata.get('comet_key')
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/developer-sidani/CLIPTrans.git

In [ ]:
%cd /content/CLIPTrans

In [ ]:
!git submodule update --init --recursive

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda env create --file environment.yml

In [ ]:
!conda env update -n base -f environment.yml

In [ ]:
PIP_PATH = '/usr/local/envs/cliptrans/bin/pip'
PYTHON_PATH= '/usr/local/envs/cliptrans/bin/python'

In [ ]:
%cd CLIPTrans

In [ ]:
!$PIP_PATH install -r requirements.txt

In [ ]:
!cd transformers && $PIP_PATH install -e .

In [ ]:
!cd data/multi30k && chmod +x setup.sh && ./setup.sh

### Pre-trained Models and Inference

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 \
    --mn multi30k --src_lang en --tgt_lang fr --prefix_length 10 \
    --bs 32 --test_ds 2016 flickr --stage translate --test \
    --lm model_best_test.pth \
    --comet_key $comet_key --comet_project CLIPTrans

### Training

#### Stage 1

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 --mn multi30k --prefix_length 10 --bs 32 --update_count 4 --lr 1e-5 --test_ds 2016 val --stage caption --tgt_lang fr

#### Stage 2

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 --mn multi30k --prefix_length 10 --bs 32 --update_count 4 --lr 1e-5 --test_ds 2016 val --stage translate --tgt_lang fr --lm model_pretrained.pth